# Restaurant contact

Scraping public restaurant adresses to get contact details. URLs have been changed for anonymity purposes.

# Import

In [155]:
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
import tqdm

from selenium import webdriver

from bs4 import BeautifulSoup
import requests

import re

## Get links for all cities

Restaurants are listed by city / region. So the first step is to get a link for all the cities.

In [39]:
url = 'https://www.----.com.gr/Rest-Greece.html'

'https://www.----.com.gr/Rest-Greece.html#LOCATION_LIST'

'https://www.tripadvisor.com.gr/Restaurants-g189398-oa20-Greece.html#LOCATION_LIST'

### First Page

The first page is different from the rest of them, so we have to handle them differently.

In [40]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [60]:
results = soup.find_all('div', class_='geo_name')

In [62]:
geo_loc_links = [r.find('a', href=True)['href'] for r in results]

### Next 10 pages

In [69]:
url_start = 'https://www.-----.com.gr/------'
url_end = '-GR.html'

In [87]:
for i in tqdm.tqdm(range(20,100,20)):
    # Sleep
    time.sleep(3)
    
    url = url_start + str(i) + url_end
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
        
    results = soup.find('ul', class_='geoList')
    results = results.find_all('a', href=True)
    geo_loc_links += [r['href'] for r in results]

100%|██████████| 4/4 [00:17<00:00,  4.27s/it]


# Load restaurants page

Choose a page to load. Here we choose the second city (Thessaloniki).

In [169]:
url_start = 'https://www.----.gr'

In [170]:
g = geo_loc_links[1]

In [171]:
url = url_start + g

## Load Driver

Go through restaurants and get each of the restaurant pages link.

In [174]:
driver = webdriver.Chrome()
driver.get(url)

In [175]:
restaurants = []
i=0
while True:
    
    try:
        time.sleep(3)

        el = driver.find_element_by_class_name('restaurants-3PzDL')
        elems = el.find_elements_by_xpath("//a[@href]")
        # Keep only Restaurant_Review
        i_rest = [elem.get_attribute("href") for elem in elems if '_Review' in elem.get_attribute("href")]

        # Keep unique values
        i_rest = list(set(i_rest))
        restaurants += i_rest

        time.sleep(2)

        el = driver.find_element_by_class_name('nav.next')
        el.click()
    except:
        break

In [178]:
pd.Series(restaurants).to_csv('restaurant_links.csv')

# Get restaurant emails

Go into each link, and get the restaurant's name and email.

In [179]:
def get_mail_from_restaurant(r):
    """Get mail from restaurant.
    
    r: restaurant link.
    returns a string with the email"""
    
    page = requests.get(r)
    soup = BeautifulSoup(page.content, 'html.parser')

    results = soup.find_all('a', href=True)

    try:
        mail = [res['href'] for res in results if 'mailto' in res['href']][0]
        mail = re.split('\?',re.sub('mailto:','',mail))[0]
    except:
        mail = ''

    return mail

In [ ]:
restaurant_emails = []

In [216]:
restaurant_emails = pd.read_csv('restaurant_emails.csv',index_col=0)

In [217]:
restaurant_emails = restaurant_emails['0'].tolist()

Get emails for the first 100 restaurants.

In [210]:
for r in tqdm.tqdm(restaurants[:100]):
    time.sleep(3)
    restaurant_emails+= [get_mail_from_restaurant(r)]







  0%|          | 0/963 [00:00<?, ?it/s]





  0%|          | 1/963 [00:04<1:14:34,  4.65s/it]





  0%|          | 2/963 [00:08<1:12:26,  4.52s/it]





  0%|          | 3/963 [00:12<1:10:19,  4.40s/it]





  0%|          | 4/963 [00:17<1:10:19,  4.40s/it]





  1%|          | 5/963 [00:21<1:09:54,  4.38s/it]





  1%|          | 6/963 [00:25<1:08:47,  4.31s/it]





  1%|          | 7/963 [00:30<1:08:03,  4.27s/it]





  1%|          | 8/963 [00:34<1:07:17,  4.23s/it]





  1%|          | 9/963 [00:38<1:07:43,  4.26s/it]





  1%|          | 10/963 [00:42<1:07:35,  4.26s/it]





  1%|          | 11/963 [00:46<1:07:14,  4.24s/it]





  1%|          | 12/963 [00:51<1:06:57,  4.22s/it]





  1%|▏         | 13/963 [00:55<1:07:41,  4.27s/it]





  1%|▏         | 14/963 [00:59<1:07:02,  4.24s/it]





  2%|▏         | 15/963 [01:03<1:07:03,  4.24s/it]





  2%|▏         | 16/963 [01:08<1:07:53,  4.30s/it]





  2%|▏         | 17/963 [01:13<1:09:33,  4.41s/it]





  2%|▏

In [211]:
pd.Series(restaurant_emails).to_csv('restaurant_emails.csv')

In [218]:
pd.Series(restaurant_emails).dropna().to_csv('restaurant_emails_final.csv')